# Space Invaders

# Weights & Biases x Qualcomm - SpaceInvaders Challenge

This notebook contains code for loading models from a file saved in a wandb run, and evaluating the model.

For more details on the SpaceInvaders challenge, please visit the [competition website](https://app.wandb.ai/wandb/spaceinvaders-challenge/benchmark/).

![](https://thumbs.gfycat.com/CookedFriendlyAntarcticfurseal-size_restricted.gif)

## Running this notebook
1. Click "Open in playground" to create a copy of this notebook for yourself.
2. Save a copy in Google Drive for yourself.
3. To enable a GPU, please click Edit > Notebook Settings. Change the "hardware accelerator" to GPU.
4. Step through each section, pressing play on the code blocks to run the cells.
5. Add your own model code.

## Load the model

Please replace the model file (`model.h5`) and run_path (`username/project_name/run_name`) with your submissions model file and run_path respectively.

In [ ]:
from __future__ import division
import os
import io
os.environ["OMP_NUM_THREADS"] = "1"
import argparse
import torch
from environment import atari_env
from utils import read_config, setup_logger
from model import A3Clstm
from player_util import Agent
import gym
import glob
import base64
import logging
import time
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from gym.wrappers import Monitor
from torchsummary import summary 

import numpy as np
from IPython.display import HTML
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display

from gym.wrappers.monitoring import video_recorder

import wandb

In [ ]:
# restore a model file from a specific run by user "lavanyashukla" in project "qualcomm" from run "mnswzdre"
fname = "model.h5"
run_path="trained_model"
local_path = None

# restore model
api = wandb.Api()
run = api.run(run_path)
with run.file(fname).download(replace=True) as f:
  local_path = f.name
local_path_model = local_path

## Setup and Preproceesing

!pip install gym pyvirtualdisplay -qq
!apt-get install -y xvfb python-opengl ffmpeg -qq
!pip install xdpyinfo -qq
!apt-get install x11-utils
!apt-get update -qq
!apt-get install cmake -qq
!pip install --upgrade setuptools -qq
!pip install ez_setup -qq

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
local_path_model = "trained_models/model.h5"

## Evaluation


In [ ]:
# **** Caution: Do not modify this cell ****
# initialize total reward across episodes
cumulative_reward = 0
episode = 0

def evaluate(episodic_reward, reset=False):
  '''
  Takes in the reward for an episode, calculates the cumulative_avg_reward
    and logs it in wandb. If episode > 100, stops logging scores to wandb.
    Called after playing each episode. See example below.

  Arguments:
    episodic_reward - reward received after playing current episode
  '''
  global episode
  global cumulative_reward
  if reset:
    cumulative_reward = 0
    episode = 0
    
  episode += 1
  print("Episode: %d"%(episode))

  # your models will be evaluated on 100-episode average reward
  # therefore, we stop logging after 100 episodes
  if (episode > 100):
    print("Scores from episodes > 100 won't be logged in wandb.")
    return

  # log total reward received in this episode to wandb
  wandb.log({'episodic_reward': episodic_reward})

  # add reward from this episode to cumulative_reward
  cumulative_reward += episodic_reward

  # calculate the cumulative_avg_reward
  # this is the metric your models will be evaluated on
  cumulative_avg_reward = cumulative_reward/episode

  # log cumulative_avg_reward over all episodes played so far
  wandb.log({'episodic_reward': episodic_reward})
  wandb.log({'cumulative_avg_reward': cumulative_avg_reward})
            
  print(episodic_reward,cumulative_avg_reward)  

  return cumulative_avg_reward

## Play the game for 100 episodes, log cumulative average reward, for 5 different values of seed

Please adjust this as needed to work with your model architecture.

In [ ]:
from numpy.random import seed

class ARGS():
    def __init__(self):
        self.max_episode_length = 100000
        self.skip_rate = 4
        self.env_config = "config.json" 

args = ARGS()

setup_json = read_config(args.env_config)
env_conf = setup_json["SpaceInvaders"]


cumulative_avg_rewards = []


saved_state = torch.load(local_path_model,map_location=lambda storage, loc: storage)

# initialize environment
env = atari_env("SpaceInvaders-v0", env_conf, args)
player = Agent(None, env, args, None)
player.model = A3Clstm(player.env.observation_space.shape[0],
                   player.env.action_space)



player.model = player.model.to(device)

player.model.load_state_dict(saved_state)
player.model.eval()

# initialize a new wandb run
wandb.init(project="qualcomm")

# define hyperparameters
wandb.config.episodes = args.max_episode_length
wandb.config.batch_size = 20
wandb.config.learning_rate = 0.0001

wandb.save("*.py")
wandb.save("*.ipynb")
wandb.save("config.json")


# record gameplay video
display = Display(visible=0, size=(1400, 900))
display.start()

player.env = gym.wrappers.Monitor(player.env, './video', force=True,mode='evaluation')


for seed_ in [10, 50, 100, 200, 500]:
  seed(seed_)
  torch.manual_seed(seed_)
  if torch.cuda.is_available():
    torch.cuda.manual_seed(seed_)
  print("Seed: ",seed_)
  episode = 0
  

  
  # run for 100 episodes
  # Note: Please adjust this as needed to work with your model architecture.
  # Make sure you still call evaluate() with the reward received in each episode
  for i in range(wandb.config.episodes):
    # Set reward received in this episode = 0 at the start of the episode
    episodic_reward = 0
    reset = False

    #vr = video_recorder.VideoRecorder(player.env,base_path=os.path.join("video","{}.{}.video".format(seed,i)))
    #vr.capture_frame()
    
    # Start a random game
    player.state = player.env.reset()
    player.state = torch.from_numpy(player.state).float().to(device)

    
    if torch.cuda.is_available():
        player.gpu_id = 0
        
    done = False
    action_count = 0

    # record a video of the game using wrapper
    #if i==50:
  
    
    while True:
        player.action_test()

        episodic_reward += player.reward
        if player.done and not player.info:
            player.state = player.env.reset()
            player.state = torch.from_numpy(player.state).float().to(device)
        elif player.info:
            break
    #print("reward",episodic_reward)
    # call evaluation function - takes in reward received after playing an episode
    # calculates the cumulative_avg_reward over 100 episodes & logs it in wandb
    if(i==0):
      reset = True

    cumulative_avg_reward = evaluate(episodic_reward, reset)

    # your models will be evaluated on 100-episode average reward
    # therefore, we stop logging after 100 episodes
    if (i >= 99):
      cumulative_avg_rewards.append(cumulative_avg_reward)
      break

    record_video = False
    
    #if i==50:
    player.env.close() 
    #vr.close()
    # render gameplay video
    if (i %50 == 0):
      mp4list = glob.glob('video/*.mp4')
      if len(mp4list) > 0:
        print(len(mp4list))
        mp4 = mp4list[-1]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)

        # log gameplay video in wandb
        wandb.log({"gameplays": wandb.Video(mp4, fps=4, format="gif")})

        # display gameplay video
        ipythondisplay.display(HTML(data='''<video alt="" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                </video>'''.format(encoded.decode('ascii'))))
player.env.close()

# Final score
The final score is evaluated as the cumulative_avg_reward, averaged across 5 seeds.

In [ ]:
print("Final score: ", np.mean(cumulative_avg_rewards))